### Problem Definition:
Three different houses each contain a different pet, a different
drink, and an inhabitant of a different nationality. The following six
facts are true about these houses:
1. The man in the third house drinks milk.

2. The Spaniard owns the dog.

3. The Ukrainian drinks the tea.

4. The Norwegian lives in the first house.

5. The Norwegian lives next to the tea-drinker.

6. The juice-drinker owns the fox.

Questions. Who owns the zebra? Who drinks juice?

### Imports:

In [11]:
from aima.logic import  PropKB, dpll_satisfiable
from aima.utils import expr,Expr

### Helping Functions

In [12]:
def symbols_combinations(symbols, nationality):
    """
    The function `symbols_combinations` takes a list of symbols and a list of nationalities, and returns
    a list of combinations where each symbol is concatenated with each nationality.
    
    :param symbols: A list of symbols
    :param nationality: A list of nationalities
    :return: a list of combinations of symbols and nationalities.
    """
    result = []
    for symbol in symbols:
        for human in nationality:
            result.append(symbol + human)
    return result

In [13]:
def general_rule_combination(symbols):
    """
    The function `general_rule_combination` takes a list of symbols and generates a set of rules based on the
    symbols' combinations.
    
    :param symbols: The `symbols` parameter is a list of strings representing nationalities. Each string
    in the list represents a nationality
    :return: a list of rules. Each rule is a string representing a logical expression in the form of a
    disjunction (OR) of conjunctions (AND).
    """
    rules = []
    count = 0
    sentence = ""

    # Each object has only one person (nationality)
    for combination in symbols:
        sentence = sentence + f"({combination}"
        for remaining in symbols:
            if combination != remaining and combination[0] == remaining[0]:
                sentence = sentence + f" & ~{remaining}"
        sentence = sentence + ")|"
        count += 1
        if count % 3 == 0:
            sentence = sentence[:-1]
            rules.append(sentence)
            sentence = ""
    count = 0
    sentence = ""

    # Each person (nationality) has only one object
    for combination in symbols:
        sentence = sentence + f"({combination}"
        for remaining in symbols:
            if combination != remaining and combination[1] == remaining[1]:
                sentence = sentence + f" & ~{remaining}"
        sentence = sentence + ")|"
        count += 1
        if count % 3 == 0:
            sentence = sentence[:-1]
            rules.append(sentence)
            sentence = ""
    return rules

In [14]:
def get_answer(results, query):
    """
    The function `get_answer` takes in a dictionary of results and a query, and returns the value from
    the results dictionary corresponding to the query.
    
    :param results: The "results" parameter is a dictionary or a data structure that contains the
    possible answers to a query. Each answer is associated with a specific key or expression
    :param query: A string representing the query or question that the user wants to ask
    :return: The function `get_answer` returns the value at the index specified by the `query` in the
    `results` list.
    """
    return results[expr(query)]

### Define combinations of Symbols:

In [15]:
pet_nationality = []
house_nationality = []
drink_nationality = []
symbols = []
# symbols
houses = ['A','B','C']
pets = ['F','Z','D']
drinks = ['M','J','T']
nationality = ['U','S','N']
# combinations of symbols
pet_nationality = symbols_combinations(pets, nationality)
house_nationality = symbols_combinations(houses, nationality)
drink_nationality = symbols_combinations(drinks, nationality)
# add combinations
symbols.extend(pet_nationality)
symbols.extend(house_nationality)
symbols.extend(drink_nationality)
symbols

['FU',
 'FS',
 'FN',
 'ZU',
 'ZS',
 'ZN',
 'DU',
 'DS',
 'DN',
 'AU',
 'AS',
 'AN',
 'BU',
 'BS',
 'BN',
 'CU',
 'CS',
 'CN',
 'MU',
 'MS',
 'MN',
 'JU',
 'JS',
 'JN',
 'TU',
 'TS',
 'TN']

### Form Knowledge base:

In [16]:
rules = []
# General Knowledge base
rules.extend(general_rule_combination(pet_nationality))
rules.extend(general_rule_combination(house_nationality))
rules.extend(general_rule_combination(drink_nationality))

# Specific Knowledge base
s1 = "(CU & MU) | (CS & MS) | (CN & MN)"
s2 = "DS"
s3 = "TU"
s4 = "AN"
s5 = ["~((BU & TU) | (BS & TS)) ==> BN", 
        "BN ==> ~((BU & TU) | (BS & TS))"]
s6 = "(JU & FU) | (JN & FN) | (JS & FS)"
rules.append(s1)
rules.append(s2)
rules.append(s3)
rules.append(s4)
rules.extend(s5)
rules.append(s6)
rules

['(FU & ~FS & ~FN)|(FS & ~FU & ~FN)|(FN & ~FU & ~FS)',
 '(ZU & ~ZS & ~ZN)|(ZS & ~ZU & ~ZN)|(ZN & ~ZU & ~ZS)',
 '(DU & ~DS & ~DN)|(DS & ~DU & ~DN)|(DN & ~DU & ~DS)',
 '(FU & ~ZU & ~DU)|(FS & ~ZS & ~DS)|(FN & ~ZN & ~DN)',
 '(ZU & ~FU & ~DU)|(ZS & ~FS & ~DS)|(ZN & ~FN & ~DN)',
 '(DU & ~FU & ~ZU)|(DS & ~FS & ~ZS)|(DN & ~FN & ~ZN)',
 '(AU & ~AS & ~AN)|(AS & ~AU & ~AN)|(AN & ~AU & ~AS)',
 '(BU & ~BS & ~BN)|(BS & ~BU & ~BN)|(BN & ~BU & ~BS)',
 '(CU & ~CS & ~CN)|(CS & ~CU & ~CN)|(CN & ~CU & ~CS)',
 '(AU & ~BU & ~CU)|(AS & ~BS & ~CS)|(AN & ~BN & ~CN)',
 '(BU & ~AU & ~CU)|(BS & ~AS & ~CS)|(BN & ~AN & ~CN)',
 '(CU & ~AU & ~BU)|(CS & ~AS & ~BS)|(CN & ~AN & ~BN)',
 '(MU & ~MS & ~MN)|(MS & ~MU & ~MN)|(MN & ~MU & ~MS)',
 '(JU & ~JS & ~JN)|(JS & ~JU & ~JN)|(JN & ~JU & ~JS)',
 '(TU & ~TS & ~TN)|(TS & ~TU & ~TN)|(TN & ~TU & ~TS)',
 '(MU & ~JU & ~TU)|(MS & ~JS & ~TS)|(MN & ~JN & ~TN)',
 '(JU & ~MU & ~TU)|(JS & ~MS & ~TS)|(JN & ~MN & ~TN)',
 '(TU & ~MU & ~JU)|(TS & ~MS & ~JS)|(TN & ~MN & ~JN)',
 '(CU & MU

### Use Propositional logic for representation

In [17]:
prop_KB = PropKB()
for rule in rules:
    prop_KB.tell(expr(rule))

### Use DPLL for Inference

In [18]:
results = dpll_satisfiable(Expr("&", *prop_KB.clauses))
results

{DS: True,
 DU: False,
 DN: False,
 FS: False,
 ZS: False,
 TU: True,
 TS: False,
 TN: False,
 MU: False,
 JU: False,
 AN: True,
 AU: False,
 AS: False,
 BN: False,
 CN: False,
 CS: True,
 CU: False,
 BU: True,
 BS: False,
 MS: True,
 MN: False,
 JS: False,
 JN: True,
 FN: True,
 FU: False,
 ZN: False,
 ZU: True}

### True symbols (for verification):
- DS
- MS
- CS
- BU
- TU
- ZU
- JN
- FN
- AN

### Ask Questions

In [19]:
# who owns zebra
print({"Ukrainian":get_answer(results,"ZU"), "Spaniard":get_answer(results,"ZS"), "Norwegian":get_answer(results,"ZN")})

# who drink juice
print({"Ukrainian":get_answer(results,"JU"), "Spaniard":get_answer(results,"JS"), "Norwegian":get_answer(results,"JN")})

{'Ukrainian': True, 'Spaniard': False, 'Norwegian': False}
{'Ukrainian': False, 'Spaniard': False, 'Norwegian': True}
